<a href="https://colab.research.google.com/github/joohwan38/DeepRL-Study/blob/main/planegame_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install stable_baselines3
!pip install gym
!pip install pygame

In [1]:
#import GYM stuff
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 

#import helpers
import numpy as np
import random
import os

#import Stable baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

#import pygame
import pygame
import time
from datetime import datetime

# pygame.quit()

pygame 2.0.1 (SDL 2.0.14, Python 3.7.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()
#오브젝트 좌표, 이속, 사이즈 초기값 세팅
class obj:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.move = 0
    def put_img(self, a, b):
        self.sx, self.sy = a, b

#충돌 판정 (y/n)
def crash(a, b):
    if (a.x-b.sx <= b.x) and (b.x <= a.x+a.sx):
        if (a.y-b.sy <= b.y) and (b.y <= a.y+a.sy):
            return True
        else:
            return False
    else:
        return False

#커스텀 환경 세팅)

class PlaneEnv(Env):
    
    def __init__(self):
        # Actions 조작의 범위 설정
        self.action_space = Discrete(3) 
        # 옵저베이션 설정 (x축, y축)
        self.observation_space = MultiDiscrete([500, 500, 900, 500, 900, 500, 900, 500, 900, 500, 900])
        # 시작 값 설정 (all zero)
        self.state = np.zeros(11, dtype=int)
        
        self.ss = obj()
        #비행선 사이즈
        self.ss.put_img(50, 60)
        
        #비행선 시작 좌표
        self.ss.x = 205 #round(size[0]/2 - self.ss.sx/2)
        self.ss.y = 825 #size[1] -self.ss.sy - 15
        self.ss.move = 20
        
        
# 1프레임마다 일어날 일돌 
    def step(self, action):
        global done
        
        #프레임당 생존 시 리워드 2점 지급
        reward =2

        #액션에 따른 비행선의 x좌표 변경 0 왼쪽 / 1 오른쪽 / 2 제자리 - 화면 가장자리를 넘어가지 않게 설정
        if action ==0:
            self.ss.x -= self.ss.move
            if self.ss.x <= 50:
                self.ss.x = 50
        elif action ==1:
            self.ss.x += self.ss.move
            if self.ss.x >= 425:
                self.ss.x = 425
        elif action ==2:
            pass

        #랜덤하게 적 비행체 생성 (프레임당 10% 확률로 생성)
        if random.random() > 0.90:
            aa = obj()
            aa.put_img(40, 40)

            # 5배의 확률로 내가 멈춰있는 x좌표 위에 적군 생성 / 나머지는 전역 생성
            mylist = [self.ss.x, random.randrange(0,478)]
            aa.x = random.choices(mylist, weights = [10, 2], k = 1)[0]
            aa.y = 10
            aa.move = 25
            a_list.append(aa)

        #레이더 설정    
        for i in range(len(a_list)):
            #y 좌표가 나보다 낮아지면 레이더 계산하지 않음
            if a_list[i].y < 825:
                pass
            #비행체의 중심부 - 적비행체 중심부간 유클리디안 거리가 100픽셀 보다 낮을때 레이더 발동
            #레이더에 걸리면, 100 나누기 둘사이 거리의 10분의 1만큼 감점 수식 (100 / (distance / 10))
            elif (abs(self.ss.x+25 - a_list[i].x+20) * abs(self.ss.y+30 - a_list[i].y+20))**(1/2) < 100:
                reward -= round(100/((abs(self.ss.x+25 - a_list[i].x+20) * abs(self.ss.y+30 - a_list[i].y+20))**(1/2)/10))

        #충돌 시 감점 로직    
        for i in range(len(a_list)):
            a = a_list[i]
            if crash(a, self.ss) == True:
                done=True
                reward -=400
        
        # 현재 프레임의 옵저베이션값 리턴 (state)
        self.state[0] = self.ss.x
        for i in range(len(a_list[:(len(self.state)-1)//2])):
            self.state[2*i+1] = a_list[i].x
            self.state[2*i+2] = a_list[i].y

        #미사일의 좌표를 화면을 넘어가면 트래킹 하지 않게 삭제해주기
        
        d_list = []
        for i in range(len(a_list)):
            ea = a_list[i]
            ea.y += ea.move
            if ea.y >= size[1]:
                d_list.append(i)
        #미사일이 화면 밖으로 넘어갔다는 것은 회피를 위미하므로, 회피시 리워드 지급하기
                reward +=30
        for d in d_list:
            del a_list[d]
        
        info = {}
        
        # Return step information
        return self.state, reward, done, info



    def render(self):
        pass
    
    #게임리셋 (환경 초기화, 비행체 시작위치 초기화)
    #적 비행체는 초기화 하지 않아도 됨 (프레임에서 생성하기 때문에 환경초기화 만으로 전체초기화 됨)
    def reset(self):
        # Reset shower temperature
        self.state = np.zeros(11, dtype=int)
        # Set shower length
        self.ss.x = 205 #round(size[0]/2 - self.ss.sx/2)
        self.ss.y = 825 #size[1] -self.ss.sy - 15
        
        return self.state

In [3]:
size = [500, 900]

env=PlaneEnv()

a_list=[]
d_list=[]


In [4]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    reward = 0
    a_list=[]
    d_list=[]
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
#         print(action, n_state, score, done)
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-368
Episode:2 Score:-328
Episode:3 Score:-328
Episode:4 Score:-347
Episode:5 Score:-445
Episode:6 Score:-320
Episode:7 Score:-294
Episode:8 Score:-398
Episode:9 Score:-304
Episode:10 Score:-331


In [5]:
log_path = os.path.join('Training', 'Logs')

In [6]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model.learn(total_timesteps=500000)#, callback=eval_callback)

Logging to Training/Logs/PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -145     |
| time/              |          |
|    fps             | 531      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1            |
|    ep_rew_mean          | -230         |
| time/                   |              |
|    fps                  | 313          |
|    iterations           | 2            |
|    time_elapsed         | 13           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0008485606 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.1         |
|    explained_variance   |

In [ ]:
#모델 저장하기
plane_path = os.path.join('Training', 'Saved Models', 'evolution_10M_PPO')

In [ ]:
model.save(plane_path)
evaluate_policy(model, env, n_eval_episodes=100, render=False)

In [ ]:
# training_log_path = os.path.join(log_path, 'PPO_20')

In [ ]:
model = PPO.load(plane_path, env)

In [ ]:
#어떤식으로 움직이고, 어떤 값을 리턴하는지 확인해보기 (랜더링 없이)

s_list=[]

episodes = 10
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    reward = 0
    a_list=[]
    d_list=[]
    
    k=0
    while not done:
        k+=1
        env.render()
        action, _states = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score+=reward
        if k %10 == 0:
          print(obs, action, score)
    s_list.append(k)
    print('Episode:{} Score:{} Total Steps:{}'.format(episode, score, k))
print('mean K:{}'.format(np.mean(s_list)))
env.close()

In [ ]:
s_list=[]

episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    reward = 0
    a_list=[]
    d_list=[]
    
    k=0
    while not done:
        k+=1
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
        # if k %10 == 0:
        #   print(obs, action, score)
    s_list.append(k)
    print('Episode:{} Score:{} Total Steps:{}'.format(episode, score, k))
print('mean K:{}'.format(np.mean(s_list)))
env.close()